# 2D projections for DNN-based reconstruction

Our DNN-based strategy for reconstruction can be described as follows:

Training:
1. Run many events in a box of pure xenon. 
2. Place the event in a random location in the active region and drift all hits to the EL plane (applying both transverse and longitudinal diffusion). Voxelize with some small resolution and store the resulting (x,y,z) coordinates and energies in an HDF5 file.
3. Slice the event in z, that is, make an x-y projection with some chosen pixel size (which may be larger than the x-y dimensions of the chosen voxel size in step 2).
4. Determine the range of all slices (the maximum extent in x and y) and eliminate slices that have a range greater than some chosen value. It is assumed that in the analysis, slices with a range greater than this value will be broken up into smaller slices that fit within the chosen range. Collect all slices that fit within the chosen range to be used as training/validation/test events.
5. For the collected slices, construct a corresponding SiPM map with a size large enough to fit the chosen range.
6. Train a DNN to reconstruct slice projections from SiPM maps; (x = [the SiPM map], y = [corresponding projection])

Analysis:
1. We begin with slices created from real data (or data-like MC). In this case we only have access to the SiPM maps and not 
2. Find active regions along the SiPM plane and extract subsets corresponding to our slice region.
3. Reconstruct the slice using the DNN.
4. Piece the track together from the reconstructed slices. The reconstructed slice coordinates will need to be translated into real (x,y) coordinates by considering where in the tracking plane the SiPM subset was taken and the voxel sizes.

* Note we may have to deal with "clashes" in slices, that is, two slices which together extend beyond the chosen DNN reconstruction range but are not separated enough to be covered by two completely independent reconstruction ranges.

In [ ]:
%pylab inline

# Sg_vs_Bg_dnn will use magbox SiPM response data to classify tracks into signal or background classes
from   __future__         import print_function
from   keras.optimizers   import SGD, Nadam
from   keras.models       import Sequential, Model
from   keras.layers       import Input, Dense, Activation, Conv2D, AveragePooling2D, MaxPooling2D, merge, Reshape, Flatten, Dropout
from   keras              import callbacks
from   matplotlib.patches import Ellipse
import matplotlib.pyplot as plt
import tables            as tb
import numpy             as np
import copy
import h5py
import sys

Define the 3D grid extent and range limits. Note that we have two potentially different voxel sizes: those in which the Geant4 events were originally voxelized ("Geant4 voxels") and those we are using for x-y projections ("projection voxels" - defined below).

In [ ]:
# Number of events to read.
nevtsTrain = 38000
nevtsTest = 2000

# SiPM plane geometry definition
nsipm = 10             # number of SiPMs in response map (a 10x10 response map covers a 100x100 range)
sipm_pitch = 10.       # distance between SiPMs
sipm_edge_width = 5.   # distance between SiPM and edge of board

# Range limit in x and y for slices (assuming a square range), in voxels
RNG_LIM = 100

gridSize = int(RNG_LIM*RNG_LIM)
mapSize = nsipm*nsipm

# Range limits from extraction of Geant4-simulated events.
NX = 200
NY = 200
NZ = 200

# Projection voxel sizes in mm
vSizeX = 1
vSizeY = 1

# The slice width, in Geant4 voxels
slice_width = 5.

# -----------------------------------------------------------------------------------------------------------
# Variables for computing an EL point location
xlen = 2*sipm_edge_width + (nsipm-1)*sipm_pitch       # (mm) side length of rectangle
ylen = 2*sipm_edge_width + (nsipm-1)*sipm_pitch       # (mm) side length of rectangle
wbin = 2.0                                            # (mm) bin width
# Compute the positions of the SiPMs
pos_x = np.ones(nsipm**2)*sipm_edge_width + (np.ones(nsipm*nsipm)*range(nsipm**2) % nsipm)*sipm_pitch
pos_y = np.ones(nsipm**2)*sipm_edge_width + np.floor(np.ones(nsipm*nsipm)*range(nsipm**2) / nsipm)*sipm_pitch


In [ ]:
# Construct a DNN.
def model_2dconv():
    
    model = Sequential()
    model.add(Reshape((1, nsipm, nsipm), input_shape=(nsipm*nsipm,)))
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='same', data_format=None))
    model.add(Conv2D(256, (2, 2), padding='same', activation='relu'))
    model.add(AveragePooling2D(pool_size=(5, 5), strides=(5, 5), padding='same', data_format=None))
    model.add(Flatten())
    model.add(Dense(units=RNG_LIM**2, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=Nadam(
        lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004))

    return model

## DNN training and reconstruction

The stored events and SiPM maps can now be read in and used to train a DNN.

In [ ]:
def read_events(file_name,nevts_train,nevts_test,map_size,grid_size):

    # Open the HDF5 file.
    evt_file = h5py.File(file_name,'r')
    tot_evts = int(len(evt_file)/3)
    
    # Make sure we have specified valid event numbers.
    if(nevts_train+nevts_test > tot_evts):
        print("ERROR: too many events specified")
        return None
    
    # Make sure we start reading test events on a boundary.
    enum_last = evt_file["evtNum{0}".format(nevts_train-1)][0]
    enum_end  = evt_file["evtNum{0}".format(nevts_train)][0]
    while(enum_last == enum_end):
        nevts_train += 1
        nevts_test  -= 1
        enum_end  = evt_file["evtNum{0}".format(nevts_train)][0]
        
    enum_last = evt_file["evtNum{0}".format(nevts_train+nevts_test-1)][0]
    enum_end  = evt_file["evtNum{0}".format(nevts_train+nevts_test)][0]
    while(enum_last == enum_end and (nevts_train+nevts_test) < tot_evts):
        nevts_test  += 1
        enum_end  = evt_file["evtNum{0}".format(nevts_train+nevts_test)][0]        
    print("Reading {0} training events and {1} test events".format(nevts_train,nevts_test))
        
    # Create the arrays to store the training and test events.
    x_train = np.zeros([nevts_train,map_size])
    x_test = np.zeros([nevts_test,map_size])
    y_train = np.zeros([nevts_train,grid_size])
    y_test = np.zeros([nevts_test,grid_size])
    
    # Start reading the events.
    print("{0} events available".format(tot_evts))
    nevents = min(tot_evts,nevts_train+nevts_test)
    for ee in range(nevents):

        # Get the data from the HDF5 file.
        slice_obj = evt_file["slice{0}".format(ee)]
        map_obj = evt_file["sipm{0}".format(ee)]
        evtnum_obj = evt_file["evtNum{0}".format(ee)]

        # Create the grid.
        xslices = slice_obj[0]; yslices = slice_obj[1]; eslices = slice_obj[2]
        slice_grid = np.zeros(gridSize)
        for xv,yv,ev in zip(xslices,yslices,eslices):
            igrid = int(xv + RNG_LIM*yv)
            slice_grid[igrid] += ev

        # Store the event.
        if(ee >= nevts_train): 
            x_test[ee-nevts_train] = map_obj
            y_test[ee-nevts_train] = slice_grid
        else:
            x_train[ee] = map_obj
            y_train[ee] = slice_grid
            
    evt_file.close()
    
    return (x_train, y_train, x_test, y_test)

## Read in the data and train the DNN

In [ ]:
x_train, y_train, x_test, y_test = read_events("data/slices.h5",nevtsTrain,nevtsTest,mapSize,gridSize)

Plot one event: grid distribution and corresponding SiPM map

In [ ]:
# Plot one event.
plt_train = True   # set to True to plot a training event, False to plot a test event
pevt = 4            # the event number
if(plt_train):
    xarr = x_train[pevt]
    yarr = y_train[pevt]
else:
    xarr = x_test[pevt]
    yarr = y_test[pevt]

# Create the figure.
fig = plt.figure();
fig.set_figheight(5.0)
fig.set_figwidth(15.0)

# Plot the SiPM response map.
ax1 = fig.add_subplot(121); 
ax1.axis([0, xlen, 0, ylen]);
probs = (xarr - min(xarr))
probs /= max(probs)
for x,y,p in zip(pos_x, pos_y, probs):
    
    # Set up the location; note we must invert y due to a pi rotation
    #  about the x-axis.
    r = Ellipse(xy=(x,y), width=2., height=2.);
    r.set_facecolor('0');
    r.set_alpha(0.02 + 0.98*p);
    ax1.add_artist(r);
ax1.set_xlabel("x (mm)")
ax1.set_ylabel("Y (mm)")

# Plot the grid.
grid_vals = np.reshape(yarr,(-1,RNG_LIM))
ax2 = fig.add_subplot(122)
extent3 = [0, RNG_LIM, 0, RNG_LIM]
sp3 = ax2.imshow(grid_vals, extent=extent3, interpolation='none', aspect='auto', origin='lower')
ax2.set_xlabel("x (mm)")
ax2.set_ylabel("Y (mm)")
cbp3 = plt.colorbar(sp3);
cbp3.set_label('Fractional Energy');

In [ ]:
# set load_model to true and specify the file to load in a previously defined/trained model
load_mdl = False
mfile = 'models/slice_reconstruction.h5'

if(load_mdl):
    model = load_model(mfile)
else:

    # otherwise define the model
    model = model_2dconv()
    
    # define callbacks (actions to be taken after each epoch of training)
    file_lbl = "{epoch:02d}-{loss:.4f}"
    filepath="weights-{0}.h5".format(file_lbl)
    checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
    tboard = callbacks.TensorBoard(log_dir='./logs', write_graph=True, write_images=False)
    lcallbacks = [checkpoint, tboard]  
model.summary()

In [ ]:
hist=model.fit(x_train, y_train, epochs=10, batch_size=50, validation_data=(x_test,y_test), callbacks=lcallbacks)

In [ ]:
# Evaluate the model and produce a list of predictions for the test data.
loss_and_metrics = model.evaluate(x_test, y_test);
y_pred = model.predict(x_test, batch_size=32, verbose=0)
print("\nMean loss is {0}".format(loss_and_metrics))

In [ ]:
# Plot true vs. reconstructed for a test event.
pevt = np.random.randint(0,1999)
pevt = 19
xarr = x_test[pevt]
y_true = y_test[pevt]
y_calc = y_pred[pevt]

# Create the figure.
fig = plt.figure();
fig.set_figheight(5.0)
fig.set_figwidth(15.0)
extent = [0, RNG_LIM, 0, RNG_LIM]

# Plot the SiPM response map.
ax1 = fig.add_subplot(121); 
ax1.axis([0, xlen, 0, ylen]);
probs = (xarr - min(xarr))
probs /= max(probs)
for x,y,p in zip(pos_x, pos_y, probs):
    
    # Set up the location; note we must invert y due to a pi rotation
    #  about the x-axis.
    r = Ellipse(xy=(x,y), width=2., height=2.);
    r.set_facecolor('0');
    r.set_alpha(0.02 + 0.98*p);
    ax1.add_artist(r);
ax1.set_xlabel("x (mm)")
ax1.set_ylabel("Y (mm)")

# Plot the truth.
ax1 = fig.add_subplot(121);
grid_true = np.reshape(y_true,(-1,RNG_LIM))
sp3 = ax1.imshow(grid_true, extent=extent, interpolation='none', aspect='auto', origin='lower')
ax1.set_xlabel("x (mm)")
ax1.set_ylabel("Y (mm)")
ax1.set_title("True Energy Distribution")
cbp3 = plt.colorbar(sp3);
cbp3.set_label('Fractional Energy');

# Plot the grid.
ax1 = fig.add_subplot(122); 
ax1.axis([0, xlen, 0, ylen]);
probs = (xarr - min(xarr))
probs /= max(probs)
for x,y,p in zip(pos_x, pos_y, probs):
    
    # Set up the location; note we must invert y due to a pi rotation
    #  about the x-axis.
    r = Ellipse(xy=(x,y), width=2., height=2.);
    r.set_facecolor('0');
    r.set_alpha(0.02 + 0.98*p);
    ax1.add_artist(r);
ax1.set_xlabel("x (mm)")
ax1.set_ylabel("Y (mm)")

ax2 = fig.add_subplot(122)
grid_calc = np.reshape(y_calc,(-1,RNG_LIM))
sp3 = ax2.imshow(grid_calc, extent=extent, interpolation='none', aspect='auto', origin='lower')
ax2.set_xlabel("x (mm)")
ax2.set_ylabel("Y (mm)")
ax2.set_title("Reconstructed Energy Distribution")
cbp3 = plt.colorbar(sp3);
cbp3.set_label('Fractional Energy');

In [ ]:
train = plt.plot(range(0,len(hist.history['loss'])),
                 np.array(hist.history['loss']), 'r', label='Train error')
val   = plt.plot(range(0,len(hist.history['loss'])),
                 np.array(hist.history['val_loss']), 'b', label='Val error')
plt.legend(loc='upper right')
plt.xlabel('Epochs')
plt.ylabel('Xentropy')
plt.title('DNN Track Reconstruction Error')
plt.grid(True)
plt.show()